In [1]:
import os
# Отключает все логи TensorFlow (0 - все, 1 - предупреждения, 2 - ошибки, 3 - критичные ошибки)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
import requests
from bs4 import BeautifulSoup
import dill
import os
import time
import re
from tqdm import tqdm
from sklearn import datasets
import pandas as pd
import numpy as np
from typing import List, Union
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, KFold
import pandas as pd
from sklearn.inspection import permutation_importance
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, IsolationForest
from sklearn.cluster import DBSCAN, KMeans
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from scipy.spatial.distance import squareform
import time
from gower import gower_matrix
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.neighbors import NearestNeighbors
from collections import Counter
from scipy.spatial.distance import pdist, squareform
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.svm import OneClassSVM
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import mutual_info_classif
import tensorflow as tf
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import torch
from torch.utils.data import Dataset, DataLoader
import time  
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from dotenv import load_dotenv
import neptune
import optuna
import time  
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay, mean_absolute_error, r2_score, mean_squared_error, make_scorer
import shap
import umap
from sklearn.manifold import TSNE
from sqlalchemy import Column, Integer, String, ForeignKey, create_engine, DateTime, func, Text, select
from sqlalchemy import Float, Boolean
from sqlalchemy.orm import declarative_base, Session, relationship, sessionmaker, configure_mappers
from sklearn.impute import SimpleImputer
import warnings
from torch.utils.data import TensorDataset, DataLoader
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, precision_score, recall_score, average_precision_score, f1_score, ndcg_score
from joblib import Parallel, delayed
import dill
from hdbscan import HDBSCAN
from sklearn.utils import resample
import plotly.express as px
from sklearn.cluster import AgglomerativeClustering
from sklearn.datasets import make_moons, make_blobs
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances
from scipy.sparse import coo_matrix, csr_matrix
from implicit.als import AlternatingLeastSquares
from sklearn.decomposition import LatentDirichletAllocation
import scipy.sparse as sp
from scipy.sparse import hstack
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Embedding, Input, Flatten
from keras.models import Model, Sequential
from keras.layers import Input, Embedding, Flatten, Dense, Concatenate
from sklearn.metrics import precision_score, recall_score, f1_score, average_precision_score, adjusted_rand_score
from keras import backend as K  # Импортируем Keras backend
from keras.utils import to_categorical
from sklearn.metrics import ndcg_score

In [3]:
print(dir(tf.keras.preprocessing))

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'image', 'image_dataset_from_directory', 'sequence', 'text', 'text_dataset_from_directory', 'timeseries_dataset_from_array']


## План работы с рекомендательными системами

**Простые модели**
- **Подготовка данных. Получение "идельных рекомендаций"**
- **1 TF-IDF + overview Базовая модель**
- **2 Сравнение метрик сходства Косинус, Jaccard, Евклид, Манхэттен**
- **3 TF-IDF + дополнительные признаки (genres, tagline)**
- **4 KNN Поиск ближайших соседей**
- **5 ALS с добавлением взаимодействия (vote_count)**
- **6 LDA (на overview) Тематическое моделирование**
- **7 Гибридная модель: TF-IDF + популярность Учет vote_average, vote_count в финальном скоринге**
  
**Нейросети**
- **8 Понижение размерности PCA/Autoencoder (на TF-IDF)  и обучение модели**
- **9 Кластеризация (Agglomerative Clustering) Группировка фильмов**
- **10 Финальный ансамбль Всё вместе: признаки + AE + кластеризация + скоринг**

## Функция для получения "идеальных рекомендаций"

In [3]:
# Путь к кэшу
CACHE_PATH = 'content/recomend/imdb_recs_cache.pkl'

# Проверка существования директории и её создание, если не существует
cache_dir = os.path.dirname(CACHE_PATH)
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)

# Загружаем кэш или создаем новый
if os.path.exists(CACHE_PATH):
    with open(CACHE_PATH, 'rb') as f:
        imdb_recs_cache = dill.load(f)
    print(f"[Загружен] Кэш на {len(imdb_recs_cache)} фильмов")
else:
    imdb_recs_cache = {}
    print("[Создан] Пустой кэш")

headers = {
    'User-Agent': 'Mozilla/5.0'
}

def get_imdb_recommendations(imdb_id, delay=2, allow_scraping=True):
    """Парсит IMDb и возвращает список рекомендованных imdb_id (если разрешено)"""

    # Уже есть в кэше — отдаем сразу
    if imdb_id in imdb_recs_cache:
        return imdb_recs_cache[imdb_id]

    # Если нельзя парсить — возвращаем пусто
    if not allow_scraping:
        return []

    url = f'https://www.imdb.com/title/{imdb_id}/'
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"[Ошибка] Не удалось загрузить страницу для {imdb_id}")
        return []

    soup = BeautifulSoup(response.text, 'html.parser')
    recs = []

    # Ищем все ссылки на похожие фильмы
    rec_block = soup.find_all('a', href=True)
    for a in rec_block:
        href = a['href']
        if href.startswith('/title/tt') and 'recommendations' not in href:
            rec_id = href.split('/')[2]
            if rec_id != imdb_id and rec_id.startswith('tt'):
                recs.append(rec_id)

    # Сохраняем только уникальные значения (до 10)
    unique_recs = list(set(recs))[:10]
    imdb_recs_cache[imdb_id] = unique_recs

    # Сохраняем кэш каждые 5 фильмов
    if len(imdb_recs_cache) % 5 == 0:
        with open(CACHE_PATH, 'wb') as f:
            dill.dump(imdb_recs_cache, f)
        print(f"[Сохранено] Кэш на {len(imdb_recs_cache)} фильмов")

    time.sleep(delay)
    return unique_recs

# Создаем ground_truth только для первых 100 фильмов
def create_ground_truth(movies_data, limit=100):
    ground_truth = {}
    
    # Ограничиваем выборку до первых `limit` фильмов
    movies_subset = movies_data.head(limit)
    
    # Для каждого фильма из subset получаем imdb_id
    for imdb_id in movies_subset['imdb_id']:
        # Получаем список рекомендаций для этого imdb_id
        recs = get_imdb_recommendations(imdb_id)
        if recs:
            # Добавляем рекомендации в ground_truth
            ground_truth[imdb_id] = recs
    
    return ground_truth


[Загружен] Кэш на 1038 фильмов


## 1. TF-IDF + overview	Базовая модель

In [7]:
# Загрузка данных
movies_metadata = pd.read_csv('content/movies_metadata.csv', low_memory=False)

In [25]:
movies_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [26]:
movies_first = movies_metadata[['title', 'overview', 'imdb_id']].dropna(subset=['title', 'overview', 'imdb_id']).reset_index(drop=True)
movies_first = movies_first[:10000]

In [27]:
movies_first

,title,overview,imdb_id
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...",tt0114709
1,Jumanji,When siblings Judy and Peter discover an encha...,tt0113497
2,Grumpier Old Men,A family wedding reignites the ancient feud be...,tt0113228
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",tt0114885
4,Father of the Bride Part II,Just when George Banks has recovered from his ...,tt0113041
...,...,...,...
9995,Él,"Francisco is rich, rather strict on principles...",tt0045361
9996,La Chienne,"Cashier Maurice Legrand is married to Adele, a...",tt0021739
9997,Eréndira,"While Erendira, a beautiful teenage girl, has ...",tt0085501
9998,The Private Lives of Elizabeth and Essex,This period drama frames the tumultuous affair...,tt0031826


In [4]:
# Путь к файлу для сохранения ground_truth
GROUND_TRUTH_PATH = 'content/recomend/ground_truth.pkl'
IMDB_RECS_CACHE_PATH = 'content/recomend/imdb_recs_cache.pkl'

In [5]:
# Проверка существования директории и её создание, если не существует
cache_dir = os.path.dirname(GROUND_TRUTH_PATH)
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)

# Загружаем или создаём ground_truth
if os.path.exists(GROUND_TRUTH_PATH):
    with open(GROUND_TRUTH_PATH, 'rb') as f:
        ground_truth = dill.load(f)
else:
    ground_truth = create_ground_truth(movies_first, limit=100)
    # Сохраняем ground_truth
    with open(GROUND_TRUTH_PATH, 'wb') as f:
        dill.dump(ground_truth, f)

# Загружаем или создаём imdb_recs_cache
if os.path.exists(IMDB_RECS_CACHE_PATH):
    with open(IMDB_RECS_CACHE_PATH, 'rb') as f:
        imdb_recs_cache = dill.load(f)
else:
    imdb_recs_cache = {}  # Создаём новый кэш

In [ ]:
# ground_truth = create_ground_truth(movies_first, limit=100)
# ground_truth_2 = create_ground_truth(movies_first, limit=1000)

In [ ]:
# ground_truth = ground_truth_2
# with open(GROUND_TRUTH_PATH, 'wb') as f:
#     dill.dump(ground_truth, f)
# with open(IMDB_RECS_CACHE_PATH, 'wb') as f:
#     dill.dump(imdb_recs_cache, f)

In [28]:
# Векторизация описаний
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(movies_first['overview'])

In [29]:
# Расчет косинусного сходства
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)


In [12]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")

# Путь к лог-файлу
_PATH = 'content/recomend/log'
os.makedirs(os.path.dirname(_PATH), exist_ok=True)

def log_to_file(message):
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    with open(_PATH, 'a', encoding='utf-8') as f:
        f.write(f"[{timestamp}] {message}\n")


# Функция для расчета регрессионных метрик
def calculate_regression_metrics(predictions, ground_truth):
    # Если в predictions или ground_truth есть строки, попробуем преобразовать их в числа
    try:
        predictions = np.array(predictions, dtype=float)
        ground_truth = np.array(ground_truth, dtype=float)
    except ValueError:
        return float('nan'), float('nan')  # Если не удается преобразовать в числа, возвращаем NaN

    if len(predictions) == 0 or len(ground_truth) == 0:
        return float('nan'), float('nan')  # Возвращаем NaN, если нет данных

    mse = mean_squared_error(ground_truth, predictions)
    rmse = mean_squared_error(ground_truth, predictions, squared=False)  # sqrt(MSE)
    return mse, rmse

def get_recommendations(title, sim, movies, movies_metadata, imdb_recs_cache, top_k=10, allow_scraping=True):
    try:
        idx_list = movies.index[movies['title'] == title].tolist()
        if not idx_list:
            log_to_file(f"⚠️ Фильм не найден в датасете: '{title}'")
            return []

        idx = idx_list[0]

        sim_scores = list(enumerate(sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:top_k+1]
        movie_indices = [i[0] for i in sim_scores]
        recommended_titles = movies['title'].iloc[movie_indices].tolist()

        rec_imdb_ids = []
        for rec_title in recommended_titles:
            meta_row = movies_metadata[movies_metadata['title'] == rec_title]
            if meta_row.empty or pd.isna(meta_row['imdb_id'].values[0]):
                log_to_file(f"ℹ️ Пропущено: нет imdb_id для '{rec_title}'")
                continue

            imdb_id = meta_row['imdb_id'].values[0]
            if imdb_id not in imdb_recs_cache:
                if not allow_scraping:
                    log_to_file(f"[Пропущено] IMDb scraping выключен для '{rec_title}' (id: {imdb_id})")
                    continue
                imdb_recs_cache[imdb_id] = get_imdb_recommendations(imdb_id, imdb_recs_cache, allow_scraping=True)

            rec_imdb_ids.extend(imdb_recs_cache.get(imdb_id, []))

        return rec_imdb_ids[:top_k]

    except Exception as e:
        log_to_file(f"❌ Ошибка при обработке '{title}': {str(e)}")
        return []

    
# Модель, возвращающая imdb_id по imdb_id
def my_recommend_model(imdb_id, movies, movies_metadata, sim, imdb_recs_cache, top_k=10, allow_scraping=True):
    title_row = movies[movies['imdb_id'] == imdb_id]
    if title_row.empty:
        return []
    title = title_row['title'].values[0]
    rec_imdb_ids = get_recommendations(title, sim, movies, movies_metadata, imdb_recs_cache, top_k, allow_scraping=allow_scraping)
    return rec_imdb_ids


# Метрики
def calculate_metrics(predictions, ground_truth):
    pred_set = set(predictions)
    true_set = set(ground_truth)
    if not pred_set or not true_set:
        return 0, 0, 0, 0, 0

    y_true = [1 if item in true_set else 0 for item in predictions]
    y_score = [1] * len(predictions)

    precision = precision_score([1]*len(y_true), y_true, zero_division=0)
    recall = recall_score([1]*len(y_true), y_true, zero_division=0)
    f1 = f1_score([1]*len(y_true), y_true, zero_division=0)
    ap = average_precision_score(y_true, y_score)

    # ✅ Проверка длины списка перед ndcg_score
    if len(y_true) > 1:
        ndcg = ndcg_score([y_true], [y_score])
    else:
        ndcg = 0.0
        log_to_file(f"NDCG пропущен из-за недостаточного количества рекомендаций: {len(y_true)}")

    return precision, recall, f1, ap, ndcg

def evaluate_model(model_func, ground_truth, movies, movies_metadata, sim, imdb_recs_cache, top_k=10, is_regression=False, allow_scraping=False):
    scores = {'precision': [], 'recall': [], 'f1': [], 'ap': [], 'ndcg': []}

    if is_regression:
        scores['mse'] = []
        scores['rmse'] = []

    for imdb_id, true_ids in tqdm(ground_truth.items(), desc="Оценка модели"):
        # Только если этот фильм есть в нашей базе
        if imdb_id not in movies['imdb_id'].values:
            continue

        # Получаем рекомендации по модели
        preds = model_func(imdb_id, movies, movies_metadata, sim, imdb_recs_cache, top_k=top_k, allow_scraping=allow_scraping)

        if is_regression:
            # Если рекомендации пусты, пропускаем этот случай
            if len(preds) == 0:
                continue
            # Если это регрессия, то вычисляем MSE и RMSE
            mse, rmse = calculate_regression_metrics(preds, true_ids)
            scores['mse'].append(mse)
            scores['rmse'].append(rmse)
        else:
            # Для классификации считаем precision, recall и другие метрики
            precision, recall, f1, ap, ndcg = calculate_metrics(preds, true_ids)
            scores['precision'].append(precision)
            scores['recall'].append(recall)
            scores['f1'].append(f1)
            scores['ap'].append(ap)
            scores['ndcg'].append(ndcg)

    # Возвращаем средние значения метрик
    result = {k: round(sum(v) / len(v), 4) if v else 0.0 for k, v in scores.items()}
    return result

def evaluate_model_als(model_func, ground_truth, movies, movies_metadata, sim, imdb_recs_cache, top_k=10, is_regression=False, allow_scraping=False):
    scores = {'precision': [], 'recall': [], 'f1': [], 'ap': [], 'ndcg': []}

    if is_regression:
        scores['mse'] = []
        scores['rmse'] = []

    for imdb_id, true_ids in tqdm(ground_truth.items(), desc="Оценка модели"):
        # Только если этот фильм есть в нашей базе
        if imdb_id not in movies['imdb_id'].values:
            continue

        # Получаем рекомендации по модели
        preds = model_func(imdb_id, movies, movies_metadata, sim, imdb_recs_cache, allow_scraping=allow_scraping)

        if is_regression:
            # Если рекомендации пусты, пропускаем этот случай
            if len(preds) == 0:
                continue
            # Если это регрессия, то вычисляем MSE и RMSE
            mse, rmse = calculate_regression_metrics(preds, true_ids)
            scores['mse'].append(mse)
            scores['rmse'].append(rmse)
        else:
            # Для классификации считаем precision, recall и другие метрики
            precision, recall, f1, ap, ndcg = calculate_metrics(preds, true_ids)
            scores['precision'].append(precision)
            scores['recall'].append(recall)
            scores['f1'].append(f1)
            scores['ap'].append(ap)
            scores['ndcg'].append(ndcg)

    # Возвращаем средние значения метрик
    result = {k: round(sum(v) / len(v), 4) if v else 0.0 for k, v in scores.items()}
    return result

In [52]:
# Загрузка ground_truth
with open('content/recomend/ground_truth.pkl', 'rb') as f:
    ground_truth = dill.load(f)

# Оценка модели
baseline_metrics = evaluate_model(my_recommend_model, ground_truth, movies_first, movies_metadata, cosine_sim, imdb_recs_cache, top_k=10)

# Вывод метрик
print(baseline_metrics)

Оценка модели: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 990/990 [00:29<00:00, 33.79it/s]

{'precision': 0.0576, 'recall': 0.0319, 'f1': 0.036, 'ap': 0.0319, 'ndcg': 0.0431}


### Метрики совсем плохии. Чтомы и видели собсвтенными глазами. Так что не удивительно.

## 2. Сравнение метрик сходства	Косинус, Jaccard, Евклид, Манхэттен

1. Косинусное сходство (Cosine Similarity)
Определение: Косинусное сходство измеряет угол между двумя векторами в многомерном пространстве. Оно не учитывает длину векторов, а только их направление. Это сходство особенно полезно при работе с текстовыми данными (например, с TF-IDF векториализацией), где важен только общий паттерн слов, а не их абсолютные частоты.

Формула:
Cosine Similarity(A, B) = (A · B) / (||A|| ||B||)
где:
A · B — скалярное произведение векторов A и B,
||A|| и ||B|| — нормы (длины) векторов A и B.
Диапазон значений: от -1 до 1
1 означает, что вектора одинаковы по направлению,
0 — что вектора ортогональны (не имеют общего направления),
-1 — что вектора противоположны.

Применение:
В основном используется для текстовых данных (например, в поисковых системах, рекомендательных системах и анализе текстов).
Хорошо работает, когда важно сравнивать схожесть вектора признаков (например, слова или документы).

2. Сходство Жаккара (Jaccard Similarity)
Определение: Сходство Жаккара измеряет схожесть двух наборов, как отношение количества общих элементов к количеству всех элементов, которые встречаются хотя бы в одном из наборов. Это мера, используемая для сравнения наборов (множеств) объектов.

Формула:
Jaccard Similarity(A, B) = |A ∩ B| / |A ∪ B|
 
где:
|A ∩ B| — количество общих элементов в множествах A и B,
|A ∪ B| — количество всех уникальных элементов в объединении множеств A и B.

Диапазон значений: от 0 до 1
1 означает, что наборы идентичны,
0 означает, что наборы не пересекаются.

Применение:
Применяется для бинарных данных, например, для сравнения наличия/отсутствия признаков (например, в рекомендательных системах для сравнения пользовательских предпочтений или товаров).
Хорошо работает для категориальных данных, где важен только факт наличия элементов, а не их количество.

3. Евклидово расстояние (Euclidean Distance)
Определение: Евклидово расстояние — это стандартная метрика для измерения прямого расстояния между двумя точками в многомерном пространстве. В отличие от косинусного сходства, оно чувствительно к разнице в величинах признаков.

Формула:

Euclidean Distance(A, B) = sqrt( Σ (A_i - B_i)^2 )
где:
A_i и B_i — элементы векторов A и B,
Σ — суммирование по всем признакам.
Диапазон значений: от 0 до ∞
0 означает, что точки совпадают,
большие значения означают большую разницу между точками.

Применение:
Используется в задачах кластеризации, например, в методах, таких как K-Means или K-Nearest Neighbors (KNN).
Хорошо работает для числовых данных, где значения признаков важны и влияют на сходство.

4. Манхэттенское расстояние (Manhattan Distance)
Определение: Манхэттенское расстояние (или таксистское расстояние) измеряет расстояние между двумя точками, двигаясь по осям (т.е. только горизонтально и вертикально, как если бы вы передвигались по прямым улицам города).

Формула:

Manhattan Distance(A, B) = Σ |A_i - B_i|
где:
A_i и B_i — элементы векторов A и B,
Σ — суммирование по всем признакам.

Диапазон значений: от 0 до ∞
0 означает, что точки совпадают,
большие значения означают большую разницу между точками.

Применение:
Используется в задачах, где важно учитывать только перемещения по осям, например, в некоторых видах кластеризации и моделях классификации.
Подходит для случаев, когда изменение в одном признаке не имеет такого же эффекта, как изменение в другом (в отличие от евклидового расстояния).

Когда использовать каждую метрику:
Косинусное сходство полезно, когда важен угол между объектами, например, в текстовых данных или когда важно понять сходство между объектами, независимо от их масштабов.
Сходство Жаккара отлично подходит для бинарных данных, когда важно учитывать только факт наличия или отсутствия признаков.
Евклидово расстояние лучше подходит для числовых данных, когда важно учитывать прямое расстояние между объектами.
Манхэттенское расстояние удобно, когда вы работаете с многими осьми, и вам важно учитывать движение вдоль отдельных осей, а не на прямую линию.

In [53]:
# Создаем список метрик сходства
similarity_metrics = {
    'cosine': cosine_similarity(tfidf_matrix),
    'euclidean': euclidean_distances(tfidf_matrix),
    'manhattan': manhattan_distances(tfidf_matrix)
}


In [54]:
# Оценка модели с различными метриками
for metric_name, sim_matrix in similarity_metrics.items():
    print(f"Оценка модели для метрики: {metric_name}")
    
    # Определяем, нужно ли использовать регрессионные метрики
    is_regression = metric_name in ['euclidean', 'manhattan']  # Для евклидова и манхэттенского расстояний используем регрессионные метрики

    # Оценка модели
    metrics_result = evaluate_model(
        my_recommend_model, 
        ground_truth, 
        movies_first, 
        movies_metadata, 
        sim_matrix, 
        imdb_recs_cache, 
        top_k=10, 
        is_regression=is_regression
    )
    
    # Вывод метрик
    print(f"Результаты для метрики {metric_name}:")
    print(metrics_result)
    print("-" * 50)

Оценка модели для метрики: cosine


Оценка модели: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 990/990 [00:29<00:00, 33.27it/s]


Результаты для метрики cosine:
{'precision': 0.0576, 'recall': 0.0319, 'f1': 0.036, 'ap': 0.0319, 'ndcg': 0.0431}
--------------------------------------------------
Оценка модели для метрики: euclidean


Оценка модели: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 990/990 [00:25<00:00, 38.51it/s]


Результаты для метрики euclidean:
{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'ap': 0.0, 'ndcg': 0.0, 'mse': nan, 'rmse': nan}
--------------------------------------------------
Оценка модели для метрики: manhattan


Оценка модели: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 990/990 [00:27<00:00, 36.24it/s]

Результаты для метрики manhattan:
{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'ap': 0.0, 'ndcg': 0.0, 'mse': 0.0, 'rmse': 0.0}
--------------------------------------------------


### Все варианты расчетов показали себя хуже косинусов. Что тоже логично т.к косинусы больше всех подхзодят к рекомендациям по текстам. Поэтому пока их и оставим

## 3 TF-IDF + дополнительные признаки (genres, tagline) "Мешок признаков"

In [55]:
movies_second = movies_metadata[['title', 'overview', 'imdb_id', 'tagline' , 'genres' ]].reset_index(drop=True)
movies_second = movies_second[:10000]

In [56]:
movies_second.head()

,title,overview,imdb_id,tagline,genres
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...",tt0114709,NaN,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,Jumanji,When siblings Judy and Peter discover an encha...,tt0113497,Roll the dice and unleash the excitement!,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,Grumpier Old Men,A family wedding reignites the ancient feud be...,tt0113228,Still Yelling. Still Fighting. Still Ready for...,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",tt0114885,Friends are the people who let you be yourself...,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,Father of the Bride Part II,Just when George Banks has recovered from his ...,tt0113041,Just When His World Is Back To Normal... He's ...,"[{'id': 35, 'name': 'Comedy'}]"


In [57]:
# 1. Объединяем все признаки в одну строку
movies_second['combined'] = movies_second['overview'] + " " + movies_second['tagline'] + " " + movies_second['genres'].apply(lambda x: ' '.join(x))

# 2. Заполняем пропущенные значения в 'combined' пустой строкой
movies_second['combined'] = movies_second['combined'].fillna('')

# 3. Векторизация всех признаков с помощью TfidfVectorizer
tfidf_vectorizer_second = TfidfVectorizer(stop_words='english', max_features=5000)
tfidf_matrix_second = tfidf_vectorizer_second.fit_transform(movies_second['combined'])

# 3. Вычисляем сходства для различных метрик
similarity_metrics_second = {
    'cosine': cosine_similarity(tfidf_matrix_second),
    'euclidean': euclidean_distances(tfidf_matrix_second),
    'manhattan': manhattan_distances(tfidf_matrix_second)
}


In [58]:
# Оценка модели с различными метриками
for metric_name, sim_matrix in similarity_metrics_second.items():
    print(f"Оценка модели для метрики: {metric_name}")
    
    # Определяем, нужно ли использовать регрессионные метрики
    is_regression = metric_name in ['euclidean', 'manhattan']  # Для евклидова и манхэттенского расстояний используем регрессионные метрики

    # Оценка модели
    metrics_result = evaluate_model(
        my_recommend_model, 
        ground_truth, 
        movies_first, 
        movies_metadata, 
        sim_matrix, 
        imdb_recs_cache, 
        top_k=10, 
        is_regression=is_regression
    )
    
    # Вывод метрик
    print(f"Результаты для метрики {metric_name}:")
    print(metrics_result)
    print("-" * 50)

Оценка модели для метрики: cosine


Оценка модели: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 990/990 [00:29<00:00, 33.93it/s]


Результаты для метрики cosine:
{'precision': 0.0071, 'recall': 0.0012, 'f1': 0.002, 'ap': 0.0012, 'ndcg': 0.0037}
--------------------------------------------------
Оценка модели для метрики: euclidean


Оценка модели: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 990/990 [00:25<00:00, 38.52it/s]


Результаты для метрики euclidean:
{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'ap': 0.0, 'ndcg': 0.0, 'mse': nan, 'rmse': nan}
--------------------------------------------------
Оценка модели для метрики: manhattan


Оценка модели: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 990/990 [00:27<00:00, 36.45it/s]

Результаты для метрики manhattan:
{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'ap': 0.0, 'ndcg': 0.0, 'mse': nan, 'rmse': nan}
--------------------------------------------------


### Пока что наша модель проста, и, возможно, для улучшения метрик потребуется переход на более сложные модели и методы, такие как использование дополнительных фичей, более мощных алгоритмов (например, нейронных сетей), а также улучшение качества данных (например, очистка данных, улучшение предобработки текста. Т.к сейчас дополнителиные фичи только ухудшили результат.

## 4. KNN Поиск ближайших соседей

KNN (K-Nearest Neighbors)
- Принцип работы:
KNN — это метод, основанный на идее поиска ближайших соседей. Для рекомендации товаров или фильмов он ищет пользователей или элементы, которые наиболее похожи на заданного пользователя или фильм. Основные шаги:
- Преобразование данных:
Мы представляем каждый фильм или пользователя в виде вектора признаков (например, с использованием TF-IDF, меток жанра или других характеристик).
Расстояние между объектами:
Чтобы найти «похожие» объекты, мы измеряем расстояние (или схожесть) между векторами с использованием таких метрик, как евклидово расстояние, косинусная схожесть и другие.
- Поиск ближайших соседей:
Для конкретного пользователя или фильма KNN находит ближайших соседей (фильмов или пользователей), основываясь на выбранной метрике сходства.
- Рекомендация:
Для фильма или пользователя на основе схожести с другими объектами рекомендатор предсказывает, какие элементы он мог бы "нравиться" или какие фильмы могут ему подойти.
Физический смысл:
Модель ищет ближайших соседей на основе сходства признаков. Например, если фильм похож на другие по жанрам, актёрам, рейтингу, то пользователю могут порекомендовать эти фильмы. KNN можно представить как способ поиск похожих объектов в большом множестве данных.
- Преимущества:
Простота и интерпретируемость.
Модель не требует обучения, потому что она просто использует данные для поиска ближайших соседей.
- Недостатки:
Высокая вычислительная нагрузка на больших данных, особенно при поиске по всей базе.
Не всегда подходит для более сложных или скрытых зависимостей между объектами.

In [59]:
k = 10  # Количество ближайших соседей

# Инициализируем модель KNN с косинусным сходством
knn = NearestNeighbors(n_neighbors=k, metric='cosine')

# Обучаем модель на tfidf_matrix
knn.fit(tfidf_matrix_second)


NearestNeighbors(metric='cosine', n_neighbors=10)

In [60]:
# Обновляем функцию для получения рекомендаций через KNN
def get_knn_recommendations(imdb_id, movies, knn_model, movies_metadata, imdb_recs_cache, top_k=10, allow_scraping=True):
    title_row = movies[movies['imdb_id'] == imdb_id]
    if title_row.empty:
        return []

    title = title_row['title'].values[0]
    idx_list = movies.index[movies['title'] == title].tolist()
    if not idx_list:
        return []
    
    idx = idx_list[0]
    distances, indices = knn_model.kneighbors(tfidf_matrix_second[idx], n_neighbors=top_k+1)

    recommended_titles = movies['title'].iloc[indices[0][1:]].tolist()  # Пропускаем первый (сам себя)
    rec_imdb_ids = []

    for rec_title in recommended_titles:
        meta_row = movies_metadata[movies_metadata['title'].str.strip().str.lower() == rec_title.strip().lower()]
        if not meta_row.empty:
            imdb_id_value = meta_row['imdb_id'].values[0]
            rec_imdb_ids.append(imdb_id_value)
        else:
            log_to_file(f"ℹ️ Пропущено: нет imdb_id для '{rec_title}'")

    return rec_imdb_ids


In [61]:
# Оценка модели с помощью KNN (без allow_scraping)
results_knn = evaluate_model(
    get_knn_recommendations,
    ground_truth,
    movies_second,
    knn,                # <--- Сначала knn модель
    movies_metadata,    # <--- Потом метаданные
    imdb_recs_cache,
    top_k=10,
    is_regression=False
)


print("Результаты оценки модели KNN:")
print(results_knn)

Оценка модели: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 990/990 [02:06<00:00,  7.80it/s]

Результаты оценки модели KNN:
{'precision': 0.0899, 'recall': 0.0117, 'f1': 0.0201, 'ap': 0.0117, 'ndcg': 0.0434}


### Результат все еще очень низкий но лучше даже чем на чистом описании. Т.е усложняя можель добавление новых признаков- всеже улучшает резульата

## 5 ALS с добавлением взаимодействия (vote_count)

ALS (Alternating Least Squares)
- Принцип работы:
ALS — это метод матричной факторизации, который активно используется для рекомендательных систем, особенно в тех случаях, когда у нас есть большая матрица взаимодействий между пользователями и объектами (например, фильмы и оценки).
- Матрица взаимодействий:
Есть большая матрица, где строки — это пользователи, а столбцы — это объекты (например, фильмы). Каждое значение в ячейке — это взаимодействие пользователя с объектом (например, оценка фильма).
- Матричная факторизация:
ALS пытается разложить эту матрицу на две меньшие матрицы: одну для пользователей и одну для объектов. Эти матрицы называются факторами.
Идея заключается в том, чтобы найти скрытые представления (факторы), которые могут объяснить взаимодействия между пользователями и объектами. Модель ищет структуру в данных и пытается понять, какие скрытые факторы (например, предпочтения пользователя, жанры фильмов) влияют на взаимодействия.
- Чередующиеся минимизации (Alternating):
ALS поочередно фиксирует одну из матриц (например, матрицу факторов пользователей) и минимизирует ошибку по другой (например, матрице факторов объектов), и наоборот. Это называется чередующейся минимизацией (alternating minimization).
Алгоритм выполняет итерации, обновляя матрицы, пока ошибка (разница между предсказанными и реальными оценками) не станет минимальной.
- Физический смысл:
Это метод выделения скрытых факторов, которые объясняют поведение пользователей. В контексте фильмов это может быть жанровые предпочтения, актерский состав, тематика или другие факторы, которые влияют на предпочтения.
Таким образом, ALS пытается понять, что пользователи «ценят» в фильмах, и на основе этого делать предсказания.
- Преимущества:
Эффективен для работы с большими наборами данных, где взаимодействий (оценок) немного.
Подходит для задач, где важно выделить скрытые зависимости между пользователями и объектами.
- Недостатки:
Требует времени на обучение и вычисления, особенно для больших данных.
Может давать плохие результаты, если данные плохо распределены или если данные разрежены.

In [62]:
movies_3 = movies_metadata[['title', 'overview', 'imdb_id', 'tagline' , 'genres', 'vote_average' , 'vote_count' ]].reset_index(drop=True)
movies_3 = movies_3[:10000]

In [63]:
# 1. Создаем фиктивных пользователей
movies_3 = movies_3.reset_index(drop=True)
movies_3['user_id'] = movies_3.index
movies_3['item_id'] = movies_3.index  # item_id тоже как индекс

# 2. Взаимодействия на основе vote_count
interactions_df = movies_3[['user_id', 'item_id', 'vote_count']].copy()

# 3. Построим sparse матрицу (user-item matrix)
user_item_matrix = coo_matrix(
    (interactions_df['vote_count'].astype(float),
     (interactions_df['user_id'], interactions_df['item_id']))
)

In [64]:
#  Шаг 1: Импорт и обучение ALS

# Увеличиваем vote_count логарифмически, чтобы не было перекоса
interactions_df['weight'] = np.log1p(interactions_df['vote_count'])

# Пересоздаем sparse матрицу с весами
user_item_matrix_weighted = coo_matrix(
    (interactions_df['weight'],
     (interactions_df['user_id'], interactions_df['item_id']))
)

# Преобразуем в формат CSR для обучения
user_item_csr = user_item_matrix_weighted.tocsr()

# Инициализируем и обучаем ALS
als_model = AlternatingLeastSquares(
    factors=50,
    regularization=0.1,
    iterations=20,
    use_gpu=False  # True если CUDA доступен
)

# Обучаем модель
als_model.fit(user_item_csr)

/home/saatarko/.conda/envs/HomeworkDS/lib/python3.12/site-packages/implicit/cpu/als.py:95: RuntimeWarning: Intel MKL BLAS is configured to use 8 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'MKL_NUM_THREADS=1' or by callng 'threadpoolctl.threadpool_limits(1, "blas")'. Having MKL use a threadpool can lead to severe performance issues
  check_blas_config()


  0%|          | 0/20 [00:00<?, ?it/s]

In [65]:
# Шаг 2: Получение рекомендаций для одного юзера
user_id = 0
recommended = als_model.recommend(user_id, user_item_csr[user_id], N=10)
print("Top-10 рекомендаций для user_0:", recommended)

Top-10 рекомендаций для user_0: (array([8180, 1508, 1258, 6562, 1640, 4112, 5553, 6740, 3183,  582],
      dtype=int32), array([0.14401034, 0.14005122, 0.13738999, 0.13507517, 0.13423133,
       0.12423028, 0.1165394 , 0.1157335 , 0.11063281, 0.10995509],
      dtype=float32))


In [99]:
# Преобразуем в csr_matrix
user_item_csr = user_item_matrix.tocsr()

def get_als_recommendations(imdb_id, movies, als_model, user_item_csr, movies_metadata, imdb_recs_cache=None, allow_scraping=False):
    # Преобразуем imdb_id в строку для безопасной работы с кэшем
    imdb_id = str(imdb_id)
    
    # Убедимся, что imdb_recs_cache имеет корректные ключи
    if not isinstance(imdb_recs_cache, dict):
        imdb_recs_cache = {}

    # Проверяем, есть ли уже рекомендации для данного imdb_id в кэше
    if imdb_id not in imdb_recs_cache:
        imdb_recs_cache[imdb_id] = []
    
    # Находим индекс фильма в DataFrame
    movie_index = movies[movies['imdb_id'] == imdb_id].index[0]
    
    # Используем индекс фильма как идентификатор пользователя (синтетический)
    user_id = movie_index  
    
    # Получаем recommendations, используя top_k из функции evaluate_model
    recommended = als_model.recommend(user_id, user_item_csr[user_id], N=11)  # 10 + 1, чтобы исключить сам фильм
    
    # recommended состоит из двух массивов: item_ids и scores
    item_ids, scores = recommended  # Это два отдельных массива
    
    # Исключаем сам фильм, если он в списке рекомендаций
    rec_ids = [item_id for item_id in item_ids if item_id != user_id]
    
    # Получаем imdb_id для рекомендуемых фильмов
    rec_imdb_ids = movies['imdb_id'].iloc[rec_ids].tolist()
    
    return rec_imdb_ids

In [100]:

# Переход к оценке модели
results_als = evaluate_model_als(
    get_als_recommendations,
    ground_truth,
    movies_3,
    als_model,
    user_item_csr,
    movies_metadata,
    imdb_recs_cache,
    is_regression=False,
    allow_scraping=False
)

print("🎯 Результаты оценки ALS:")
print(results_als)


Оценка модели: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 990/990 [00:05<00:00, 171.44it/s]

🎯 Результаты оценки ALS:
{'precision': 0.0273, 'recall': 0.0025, 'f1': 0.0045, 'ap': 0.0025, 'ndcg': 0.012}


### Полученные метрики хуже чем у KNN

## 6. LDA (на overview) Тематическое моделирование

LDA (Latent Dirichlet Allocation) — это алгоритм для тематического моделирования, который используется для выделения скрытых тем в текстовых данных. Он помогает понять, какие темы присутствуют в коллекции документов, и какие слова наиболее характерны для этих тем. LDA является вероятностной моделью, которая основывается на статистическом подходе.

- Принцип работы LDA:
LDA — это генеративная модель, которая предполагает следующее:
Каждый документ (например, обзор фильма) является смесью нескольких тем.
Каждая тема является распределением по словам (например, слова, связанные с любовью, драмой или фантастикой для фильмов).
Задача LDA заключается в том, чтобы выявить эти скрытые темы, которые, вероятно, объясняют содержание документа.
Алгоритм состоит из следующих этапов:

- Предположения:
Каждый документ представляет собой смесь нескольких скрытых тем.
Каждая тема представляется распределением по словам.
Процесс генерации документа (генеративная модель):
Выбирается тема (из заранее заданного количества тем K).
Для каждой темы выбирается слово из темы в соответствии с вероятностным распределением для этой темы.
Этот процесс повторяется для всех слов в документе.
- Обучение модели:
Алгоритм пытается найти наиболее вероятные темы, которые могли бы привести к набору слов в документах. Это происходит через итерации, где на каждом шаге алгоритм обновляет вероятности того, какая тема порождает какое слово в документе, и каким образом темы распределяются по документам.
Распределение по темам: После обучения модели, для каждого документа мы получаем распределение тем — то есть, сколько процентов каждого из K возможных тем есть в этом документе. Это позволяет выявить ключевые темы каждого документа (например, фильмы о любви, научной фантастике, комедии).
- Ключевые элементы в LDA:
Темы (Topics) — наборы слов, которые часто встречаются вместе в текстах. Каждая тема характеризуется распределением вероятности по словам (например, для темы "спорт" это будут слова: "игра", "команда", "футбол" и т.д.).
Документы (Documents) — наборы слов. Документ может содержать несколько тем с разными вероятностями (например, обзор фильма может быть смесью тем "драма" и "приключения").
Алгоритм — LDA использует методы скрытого распределения и Байесовскую индукцию для нахождения скрытых тем в тексте.
- Зачем использовать LDA:
Поиск скрытых тем в текстах: LDA помогает анализировать текстовые данные, извлекая темы, которые могут быть важными для понимания содержимого документа или коллекции документов. Это полезно, например, для классификации текстов по темам или улучшения поисковых систем.
Снижение размерности: Множество текстов, например, 5000 слов в обзоре фильма, может быть сведено к нескольким скрытым темам, что позволяет уменьшить количество признаков и улучшить обработку данных для дальнейших задач, таких как кластеризация или рекомендации.
Выявление паттернов: Это помогает находить связи и паттерны в данных, которые не очевидны на первый взгляд.
- Пример:
Предположим, у нас есть коллекция фильмов с их обзорами. С помощью LDA мы можем выделить несколько тем, например:
Тема 1: Любовь, романтика, отношения (слова: любовь, роман, отношения, сердце)
Тема 2: Приключения, экшн (слова: приключения, опасность, борьба, спасение)
Тема 3: Научная фантастика, будущее (слова: космос, технологии, будущее, исследование)
Каждый фильм будет иметь распределение этих тем, например:
Фильм 1: 70% "Любовь", 20% "Приключения", 10% "Научная фантастика"
Фильм 2: 10% "Любовь", 60% "Приключения", 30% "Научная фантастика"
Так, LDA помогает нам понять, что скрыто в текстах, и создать более осмысленные рекомендации или аналитику.
- Важные параметры в LDA:
n_components — количество тем, которое вы хотите получить. Это параметр, который вы выбираете до начала обучения модели.
max_iter — количество итераций для обучения модели.
learning_decay и learning_offset — параметры, контролирующие скорость обучения модели.

In [101]:
# 1. Подготовка данных: выберем обзоры
overviews = movies_3['overview'].dropna()

# 2. Преобразование текста в TF-IDF матрицу
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
tfidf_matrix = tfidf_vectorizer.fit_transform(overviews)

# 3. Обучение модели LDA
lda = LatentDirichletAllocation(n_components=10, random_state=42)  # выбрано 10 тем
lda_topics = lda.fit_transform(tfidf_matrix)

# 4. Посмотрим на топ слова для каждой темы
n_words = 10  # количество слов для вывода в каждой теме
feature_names = tfidf_vectorizer.get_feature_names_out()

for topic_idx, topic in enumerate(lda.components_):
    print(f"Тема #{topic_idx + 1}:")
    print(" ".join([feature_names[i] for i in topic.argsort()[:-n_words - 1:-1]]))
    print()

# 5. Создаем новые признаки для фильмов на основе тем
topic_columns = [f'topic_{i+1}' for i in range(lda.n_components)]
topics_df = pd.DataFrame(lda_topics, columns=topic_columns)

# 6. Добавляем их в исходный DataFrame
movies_3 = pd.concat([movies_3, topics_df], axis=1)

print(movies_3.head())

Тема #1:
war film young life world story man love french new

Тема #2:
new detective agent police killer young murder cop york secret

Тема #3:
king arthur story love young based prince throne novel clerk

Тема #4:
love young man story family stolen new life film american

Тема #5:
documentary film life godzilla music comedy town story world young

Тема #6:
war world film man young group evil new life story

Тема #7:
life love young man family new woman father wife mother

Тема #8:
world war crew mysterious planet british film ship ii american

Тема #9:
life new school young man high old drug town love

Тема #10:
germany nazi justice communist hitler killer chip eugene dante king

                         title  \
0                    Toy Story   
1                      Jumanji   
2             Grumpier Old Men   
3            Waiting to Exhale   
4  Father of the Bride Part II   

                                            overview    imdb_id  \
0  Led by Woody, Andy's toys live happ

In [169]:
def get_lda_recommendations(imdb_id, movies, movies_metadata, imdb_recs_cache, top_k=10, allow_scraping=False):
    movie_index = movies[movies['imdb_id'] == imdb_id].index[0]
    movie_overview = movies.loc[movie_index, 'overview']
    
    # Преобразуем обзор фильма в TF-IDF вектор
    movie_tfidf = tfidf_vectorizer.transform([movie_overview])  # Используем глобальный tfidf_vectorizer
    
    # Получаем распределение тем для этого фильма
    movie_topics = lda.transform(movie_tfidf)[0]
    
    # Получаем распределения тем для всех фильмов (перед тем, как вычислять сходство)
    all_movie_topics = lda.transform(tfidf_vectorizer.transform(movies['overview'].dropna()))
    
    # Получаем схожесть между текущим фильмом и всеми остальными фильмами
    similarity_scores = cosine_similarity([movie_topics], all_movie_topics)[0]

    # Сортируем индексы по убыванию
    recommended_idx = similarity_scores.argsort()[::-1]
    
    # Убедимся, что recommended_idx имеет тип numpy.ndarray
    recommended_idx = np.array(recommended_idx)
    
    # Получаем топ K элементов
    recommended_idx = recommended_idx[:10]
    
    # Приводим тип индексов к int, если необходимо
    recommended_idx = recommended_idx.astype(int)

    # Извлекаем фильмы с помощью индексов
    recommended_movies = movies.iloc[recommended_idx]
    
    return recommended_movies['imdb_id'].tolist()

# Обертка для метода get_lda_recommendations
def get_lda_recommendations_with_tfidf(imdb_id, movies, movies_metadata, imdb_recs_cache, top_k=10, allow_scraping=False, tfidf_vectorizer=None):
    # Просто передаем tfidf_vectorizer в глобальную область
    global tfidf_vectorizer_global
    tfidf_vectorizer_global = tfidf_vectorizer
    
    return get_lda_recommendations(
        imdb_id, 
        movies, 
        movies_metadata, 
        imdb_recs_cache, 
        top_k=top_k, 
        allow_scraping=allow_scraping
    )

In [170]:
results_lda = evaluate_model_als(
    get_lda_recommendations_with_tfidf,  # Используем обертку
    ground_truth,
    movies_3,
    movies_metadata,
    user_item_csr,
    imdb_recs_cache,
    top_k=10,
    is_regression=False
)

print("🎯 Результаты оценки LDA:")
print(results_lda)

Оценка модели: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 990/990 [21:13<00:00,  1.29s/it]

🎯 Результаты оценки LDA:
{'precision': 0.0162, 'recall': 0.0016, 'f1': 0.0029, 'ap': 0.0016, 'ndcg': 0.0073}


### Метрки хуже чем у KNN т.к опять же мтеод расчитан на более серьезную модель

## 7 Гибридная модель: TF-IDF + популярность Учет vote_average, vote_count в финальном скоринге

Гибридная модель: TF-IDF + популярность:
- 1. TF-IDF матрица:
Это tfidf_matrix, например, [10000 x 5000], где каждая строка — это фильм, а каждая колонка — слово из описания.
Она отвечает за смысловую близость.
- 2. Популярность:
Мы создаём числовой показатель, например:
popularity_score = vote_average * log1p(vote_count)
Он говорит, насколько фильм хорош по мнению аудитории.
Мы нормализуем этот скор, чтобы он был в [0, 1], и добавляем как дополнительный признак к TF-IDF.
- 3. Объединение признаков:
С помощью scipy.sparse.hstack() мы объединяем sparse TF-IDF и колонку популярности:
combined_matrix = hstack([alpha * tfidf_matrix, beta * popularity_matrix])
Это новая матрица [10000 x (5000+1)].
- 4. Косинусное сходство:
Теперь мы меряем похожесть фильмов не только по смыслу, но и с поправкой на популярность:
cosine_sim = linear_kernel(combined_matrix, combined_matrix)
Это улучшает рекомендации: фильмы будут и похожие по теме, и любимые зрителями.
- Почему это работает:
TF-IDF может вернуть очень редкие, неочевидные фильмы.
Популярность фильтрует откровенно слабые и "нишевые".
Гибрид балансирует между "релевантный по смыслу" и "популярный/оценённый".
- Можно тюнинговать веса:
alpha = 0.9  # смысл важнее
beta = 0.1   # но не забываем про качество
📊 Пример:
Ищем рекомендации к фильму "The Lion King":

Только TF-IDF: может вернуть фильм про "джунгли", "царей", но с рейтингом 3.5 и 200 голосов.
Только популярность: может вернуть любой популярный фильм, даже не по теме.
Гибрид: даст и тематически близкий, и качественный.



In [8]:
movies_3 = movies_metadata[['title', 'overview', 'imdb_id', 'tagline' , 'genres', 'vote_average' , 'vote_count' ]].reset_index(drop=True)
movies_3 = movies_3[:10000]

In [9]:
# ===== 🔧 Настройки =====
alpha = 0.8  # Вес TF-IDF
beta = 0.2   # Вес популярности
top_k = 10

# ===== 📥 Загрузка и подготовка данных =====
movies_3 = movies_3.dropna(subset=['overview']).copy()
movies_3['vote_count'] = pd.to_numeric(movies_3['vote_count'], errors='coerce').fillna(0)
movies_3['vote_average'] = pd.to_numeric(movies_3['vote_average'], errors='coerce').fillna(0)
movies_3['popularity_score'] = movies_3['vote_average'] * np.log1p(movies_3['vote_count'])

# ===== 🧪 TF-IDF =====
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies_3['overview'])

# ===== 🔢 Нормализация популярности и объединение =====
popularity_norm = (movies_3['popularity_score'] - movies_3['popularity_score'].min()) / (movies_3['popularity_score'].max() - movies_3['popularity_score'].min())
popularity_matrix = csr_matrix(popularity_norm.values[:, np.newaxis])  # sparse column

# Объединяем TF-IDF и популярность как sparse матрицы
combined_matrix = hstack([alpha * tfidf_matrix, beta * popularity_matrix])

# ===== 🧠 Косинусная близость =====
cosine_sim = linear_kernel(combined_matrix, combined_matrix)

# ===== 🔄 Индексы =====
index_to_id = dict(enumerate(movies_3['imdb_id']))
id_to_index = {v: k for k, v in index_to_id.items()}

In [15]:
# ===== 🔍 Гибридная модель =====
def hybrid_model_func(imdb_id, movies, movies_metadata, imdb_recs_cache=None, top_k=10, allow_scraping=False):
    if imdb_id not in id_to_index:
        print(f"⚠️ IMDb ID не найден: {imdb_id}")
        return []
    idx = id_to_index[imdb_id]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = [s for s in sim_scores if s[0] != idx][:10]
    rec_indices = [i[0] for i in sim_scores]
    rec_ids = [index_to_id[i] for i in rec_indices]
    return rec_ids

In [16]:
results = evaluate_model_als(
    model_func=hybrid_model_func,
    ground_truth=ground_truth,
    movies=movies_3,
    movies_metadata=movies_3,
    sim=None,
    imdb_recs_cache=imdb_recs_cache,
    top_k=10,
    is_regression=False,
    allow_scraping=False
)

print("📊 Результаты гибридной модели:", results)

Оценка модели: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 990/990 [00:11<00:00, 84.57it/s]

📊 Результаты гибридной модели: {'precision': 0.1414, 'recall': 0.0188, 'f1': 0.0325, 'ap': 0.0188, 'ndcg': 0.0688}


In [ ]:
# ===== 2. Популярность: vote_average * log(vote_count) =====
vote_count = movies_3['vote_count'].fillna(0)
vote_average = movies_3['vote_average'].fillna(0)
popularity_score = vote_average * np.log1p(vote_count)

# Масштабируем
scaler = MinMaxScaler()
popularity_scaled = scaler.fit_transform(popularity_score.values.reshape(-1, 1))

# ===== 3. Гибрид: TF-IDF + популярность =====
# Трансформируем популярность в sparse формат такой же формы, как tfidf_matrix
from scipy.sparse import csr_matrix
popularity_sparse = csr_matrix(np.repeat(popularity_scaled, tfidf_matrix.shape[1], axis=1))

combined_matrix = alpha * tfidf_matrix + beta * popularity_sparse

# ===== 4. PCA =====
pca = PCA(n_components=2)
reduced = pca.fit_transform(combined_matrix.toarray())

# ===== 5. Визуализация =====
plt.figure(figsize=(12, 8))
plt.scatter(reduced[:, 0], reduced[:, 1],
            c=popularity_scaled.flatten(),  
            cmap='viridis', s=20, alpha=0.7)
plt.colorbar(label='Популярность (норм.)')
plt.title('📽️ Визуализация фильмов: TF-IDF + Популярность')
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.tight_layout()
plt.show()

### Самый улчшие пока на данный момент метрики. Что также не удивительно т.к это игбриный метод. т.е сочеатние несоклких методов

## 8  Понижение размерности PCA/Autoencoder (на TF-IDF) и обучение модели

Шаг 1: Подготовка данных

In [6]:
# Загрузка данных
movies_metadata = pd.read_csv('content/movies_metadata.csv', low_memory=False)

# Обрезаем до первых 10000 строк для уменьшения нагрузки на память
movies_metadata = movies_metadata[:10000]

# Преобразуем числовые признаки с помощью Min-Max нормализации
scaler = MinMaxScaler()

# Преобразуем 'vote_average', 'popularity', 'vote_count' в числовой формат
movies_metadata['vote_average'] = pd.to_numeric(movies_metadata['vote_average'], errors='coerce')
movies_metadata['popularity'] = pd.to_numeric(movies_metadata['popularity'], errors='coerce')
movies_metadata['vote_count'] = pd.to_numeric(movies_metadata['vote_count'], errors='coerce')

# Применяем Min-Max нормализацию
movies_metadata[['vote_average', 'popularity', 'vote_count']] = scaler.fit_transform(
    movies_metadata[['vote_average', 'popularity', 'vote_count']])

# Преобразование категориальных признаков (например, genres и tagline)
movies_metadata['genres'] = movies_metadata['genres'].apply(lambda x: x.split(',') if isinstance(x, str) else [])
movies_metadata['tagline'] = movies_metadata['tagline'].fillna('')

# Преобразование tagline в числовые признаки с использованием CountVectorizer
count_vectorizer = CountVectorizer(max_features=10000, stop_words='english')
tagline_features = count_vectorizer.fit_transform(movies_metadata['tagline'])

# Преобразование overview в TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
overview_tfidf = tfidf_vectorizer.fit_transform(movies_metadata['overview'].fillna(''))

# Получаем список уникальных жанров
all_genres = movies_metadata['genres'].dropna().explode().unique()

# Создаем словарь для индексации жанров
genre_to_index = {genre: idx for idx, genre in enumerate(all_genres)}

# Преобразуем жанры в индексы
movies_metadata['genre_indices'] = movies_metadata['genres'].apply(lambda genres: [genre_to_index[genre] for genre in genres])

# Подготовка числовых признаков
numerical_features = np.hstack([movies_metadata[['vote_average', 'popularity', 'vote_count']].values, tagline_features.toarray()])
final_features = np.hstack([numerical_features, overview_tfidf.toarray()])

In [7]:
# Разделение данных на обучающую и тестовую выборки
X_train, X_test = train_test_split(final_features, test_size=0.2, random_state=42)

In [8]:
final_features.shape

(10000, 11817)

Шаг 2: Создание модели Autoencoder

In [12]:
# Определение модели Autoencoder
input_layer = Input(shape=(X_train.shape[1],))

# Кодировщик (сжатие данных)
encoded = Dense(500, activation='relu')(input_layer)
encoded = Dense(100, activation='relu')(encoded)  # Сжимающий слой

# Декодировщик (восстановление данных)
decoded = Dense(500, activation='relu')(encoded)
decoded = Dense(X_train.shape[1], activation='sigmoid')(decoded)  # Восстановление до исходного размера

# Модель Autoencoder
autoencoder = Model(input_layer, decoded)

# Компиляция модели
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Обучение модели Autoencoder
autoencoder.fit(X_train, X_train, epochs=50, batch_size=256, shuffle=True, validation_data=(X_test, X_test))

# Получаем сжатые данные
encoder = Model(input_layer, encoded)  # Модель для получения сжатых признаков
X_train_encoded = encoder.predict(X_train)
X_test_encoded = encoder.predict(X_test)

print("Размерность обучающей выборки после сжатия:", X_train_encoded.shape)

Epoch 1/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 9s 234ms/step - loss: 0.2169 - val_loss: 3.7232e-04
Epoch 2/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 7s 217ms/step - loss: 3.6673e-04 - val_loss: 3.6297e-04
Epoch 3/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 7s 219ms/step - loss: 3.7264e-04 - val_loss: 3.6291e-04
Epoch 4/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 7s 220ms/step - loss: 3.7387e-04 - val_loss: 3.6288e-04
Epoch 5/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 7s 215ms/step - loss: 3.7038e-04 - val_loss: 3.6288e-04
Epoch 6/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 7s 226ms/step - loss: 3.7251e-04 - val_loss: 3.6293e-04
Epoch 7/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 7s 218ms/step - loss: 3.7420e-04 - val_loss: 3.6302e-04
Epoch 8/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 7s 220ms/step - loss: 3.7238e-04 - val_loss: 3.6288e-04
Epoch 9/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 7s 217ms/step - loss: 3.6940e-04 - val_loss: 3.6289e-04
Epoch 10/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 7s 219ms/step - loss: 3.7092e-04 - val_loss: 3.6290e-04
Epoch 11/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 7s 223ms/step - loss: 3.

In [13]:
# # Путь к файлу для сохранения 
# X_TRAIN_ENCODE_PATH = 'content/recomend/X_train_encoded.pkl'
# X_TEST_ENCODE_PATH = 'content/recomend/X_test_encoded.pkl'

# # Проверка существования директории и её создание, если не существует
# cache_dir = os.path.dirname(X_TRAIN_ENCODE_PATH)
# if not os.path.exists(cache_dir):
#     os.makedirs(cache_dir)
# with open(X_TRAIN_ENCODE_PATH, 'wb') as f:
#     dill.dump(X_train_encoded, f)

# # Проверка существования директории и её создание, если не существует
# cache_dir = os.path.dirname(X_TEST_ENCODE_PATH)
# if not os.path.exists(cache_dir):
#     os.makedirs(cache_dir)
# with open(X_TEST_ENCODE_PATH, 'wb') as f:
#     dill.dump(X_test_encoded, f)

In [9]:
X_TRAIN_ENCODE_PATH = 'content/recomend/X_train_encoded.pkl'
with open(X_TRAIN_ENCODE_PATH, 'rb') as f:
        X_train_encoded = dill.load(f)

X_TEST_ENCODE_PATH = 'content/recomend/X_test_encoded.pkl'
with open(X_TEST_ENCODE_PATH, 'rb') as f:
        X_test_encoded = dill.load(f)

Шаг 3: Обучение модели на сжатых данных и предсказания

In [50]:
# ground_truth

In [13]:
# Возьмем только те фильмы, для которых есть ground_truth (идеальные рекомендации)
ground_truth_keys = list(ground_truth.keys())

# Создание бинарных меток для фильмов в ground_truth
def create_binary_vector(imdb_id, ground_truth_keys, ground_truth):
    return [1 if movie_id in ground_truth.get(imdb_id, []) else 0 for movie_id in ground_truth_keys]


# Разделение на обучающую и тестовую выборки, учитывая фильмы из ground_truth
train_indices = [i for i, imdb_id in enumerate(movies_metadata['imdb_id'][:10000]) if imdb_id in ground_truth_keys]
test_indices = [i for i, imdb_id in enumerate(movies_metadata['imdb_id'][:10000]) if imdb_id in ground_truth_keys and i not in train_indices]

# Если в test_indices пусто, можно перенести часть фильмов из train_indices в test_indices
if len(test_indices) == 0:
    # Допустим, перемещаем 10 фильмов из train в test
    test_indices = train_indices[:10]
    train_indices = train_indices[10:]

# Извлекаем только фильмы, которые есть в ground_truth
X_train_filtered = X_train_encoded[train_indices]
X_test_filtered = X_test_encoded[test_indices]

# Проверка размерностей
print(f"Размер X_train_filtered: {X_train_filtered.shape}")
print(f"Размер X_test_filtered: {X_test_filtered.shape}")

# Создание бинарных меток для фильмов в ground_truth
y_train_filtered = np.array([create_binary_vector(movies_metadata['imdb_id'][i], ground_truth_keys, ground_truth) for i in train_indices])
y_test_filtered = np.array([create_binary_vector(movies_metadata['imdb_id'][i], ground_truth_keys, ground_truth) for i in test_indices])

# Проверка размерностей
print(f"Размер y_train_filtered: {y_train_filtered.shape}")
print(f"Размер y_test_filtered: {y_test_filtered.shape}")

# Преобразуем y_train и y_test в one-hot формат
y_train_one_hot = y_train_filtered
y_test_one_hot = y_test_filtered


Размер X_train_filtered: (981, 100)
Размер X_test_filtered: (10, 100)
Размер y_train_filtered: (981, 990)
Размер y_test_filtered: (10, 990)


In [14]:
# Функция для вычисления NDCG
def ndcg_at_k(y_true, y_pred, k=10):
    order = np.argsort(y_pred, axis=1)[:, ::-1]  # Порядок сортировки предсказанных значений
    y_true = np.array(y_true)
    dcg = np.sum((y_true[np.arange(y_true.shape[0]), order[:, :k]] > 0).astype(int) / np.log2(np.arange(2, k + 2)))
    idcg = np.sum((y_true[np.arange(y_true.shape[0]), order[:, :k]] > 0).astype(int) / np.log2(np.arange(2, k + 2)))
    return np.mean(dcg / idcg)  # Нормализованный DCG

# Определение кастомных метрик для Keras
def precision(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)  # Приводим y_true к типу float32
    y_pred = tf.cast(y_pred, tf.float32)  # Приводим y_pred к типу float32
    return tf.reduce_mean(tf.cast(tf.equal(tf.round(y_true), tf.round(y_pred)), tf.float32))

def recall(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    return tf.reduce_mean(tf.cast(tf.equal(tf.round(y_true), tf.round(y_pred)), tf.float32))

def f1(y_true, y_pred):
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    return 2 * ((p * r) / (p + r + tf.keras.backend.epsilon()))

In [19]:
# Создание модели
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=X_train_filtered.shape[1]))  # Входной слой с меньшим количеством нейронов
model.add(Dense(32, activation='relu'))  # Промежуточный слой
model.add(Dense(y_train_one_hot.shape[1], activation='sigmoid'))

model.compile(optimizer='adam', 
              loss='binary_crossentropy',
              metrics=['accuracy', precision, recall, f1])


early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# Обучение модели с меньшим количеством эпох
X_train_filtered_small = X_train_filtered[:100]
y_train_one_hot_small = y_train_one_hot[:100]

model.fit(X_train_filtered, y_train_one_hot, epochs=100, batch_size=32, validation_data=(X_test_filtered, y_test_one_hot), callbacks=[early_stopping])


# Оценка модели
loss, accuracy, precision_val, recall_val, f1_val = model.evaluate(X_test_filtered, y_test_one_hot)
print(f'Точность: {accuracy}')
print(f'Precision: {precision_val}')
print(f'Recall: {recall_val}')
print(f'F1: {f1_val}')



Epoch 1/100


/home/saatarko/.conda/envs/HomeworkDS/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.0000e+00 - f1: 0.6977 - loss: 0.5835 - precision: 0.6977 - recall: 0.6977 - val_accuracy: 0.0000e+00 - val_f1: 0.9946 - val_loss: 0.0787 - val_precision: 0.9946 - val_recall: 0.9946
Epoch 2/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - f1: 0.9978 - loss: 0.0381 - precision: 0.9978 - recall: 0.9978 - val_accuracy: 0.0000e+00 - val_f1: 0.9977 - val_loss: 0.0175 - val_precision: 0.9977 - val_recall: 0.9977
Epoch 3/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0000e+00 - f1: 0.9984 - loss: 0.0143 - precision: 0.9984 - recall: 0.9984 - val_accuracy: 0.0000e+00 - val_f1: 0.9977 - val_loss: 0.0164 - val_precision: 0.9977 - val_recall: 0.9977
Epoch 4/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 5.5187e-04 - f1: 0.9985 - loss: 0.0129 - precision: 0.9985 - recall: 0.9985 - val_accuracy: 0.0000e+00 - val_f1: 0.9977 - val_loss: 0.0155 - val_precision: 0.9977 - val_recall: 0.9977
Epoch 5/100
31/31 ━━━━━

In [ ]:
# Предположим, что они хранятся в `X_additional_features`
X_additional_features = np.array([...])  # Пример дополнительных признаков

# Объединяем сжатыми признаками из Autoencoder и дополнительными признаками
X_train_encoded_combined = np.concatenate([X_train_encoded, X_additional_features[:X_train_encoded.shape[0]]], axis=1)
X_test_encoded_combined = np.concatenate([X_test_encoded, X_additional_features[X_train_encoded.shape[0]:]], axis=1)

# Проверка размерности
print("Размерность X_train после объединения:", X_train_encoded_combined.shape)
print("Размерность X_test после объединения:", X_test_encoded_combined.shape)

In [25]:
k = 10  # топ-k рекомендаций

eval_imdb_ids = list(ground_truth.keys())
precisions = []
recalls = []

for imdb_id in eval_imdb_ids:
    try:
        # Находим индекс фильма
        idx = movies_metadata[movies_metadata['imdb_id'] == imdb_id].index[0]
        
        # Получаем вектор признаков и предсказания
        x_input = X_train_encoded[idx].reshape(1, -1)
        
        # Получение предсказаний без вывода
        predicted_scores = model.predict(x_input, verbose=0).flatten()

        # Получаем индексы top-k предсказанных фильмов
        top_k_indices = np.argsort(predicted_scores)[-k:][::-1]
        predicted_ids = [movies_metadata['imdb_id'][i] for i in top_k_indices]

        # Ground truth для текущего фильма
        true_recs = ground_truth[imdb_id]

        # Binary vector: 1, если фильм есть в ground_truth
        y_true = [1 if imdb in true_recs else 0 for imdb in predicted_ids]

        # Метрики
        precisions.append(np.sum(y_true) / k)
        recalls.append(np.sum(y_true) / len(true_recs))  # обычно 10

    except Exception as e:
        print(f"Ошибка для {imdb_id}: {e}")
        continue

print(f"\n📊 Средний Precision@{k}: {np.mean(precisions):.4f}")
print(f"📊 Средний Recall@{k}: {np.mean(recalls):.4f}")


📊 Средний Precision@10: 0.0041
📊 Средний Recall@10: 0.0041


### Показатели очень слабые но сам энкодер былсделан без подора параметров

In [ ]:
Шаг 4: Кластеризация на сжатых данных

In [6]:
# Загрузка данных
movies_metadata = pd.read_csv('content/movies_metadata.csv', low_memory=False)

# Обрезаем до первых 10000 строк для уменьшения нагрузки на память
movies_metadata = movies_metadata[:10000]

# Преобразуем числовые признаки с помощью Min-Max нормализации
scaler = MinMaxScaler()

# Преобразуем 'vote_average', 'popularity', 'vote_count' в числовой формат
movies_metadata['vote_average'] = pd.to_numeric(movies_metadata['vote_average'], errors='coerce')
movies_metadata['popularity'] = pd.to_numeric(movies_metadata['popularity'], errors='coerce')
movies_metadata['vote_count'] = pd.to_numeric(movies_metadata['vote_count'], errors='coerce')

# Применяем Min-Max нормализацию
movies_metadata[['vote_average', 'popularity', 'vote_count']] = scaler.fit_transform(
    movies_metadata[['vote_average', 'popularity', 'vote_count']])

# Преобразование категориальных признаков (например, genres и tagline)
movies_metadata['genres'] = movies_metadata['genres'].apply(lambda x: x.split(',') if isinstance(x, str) else [])
movies_metadata['tagline'] = movies_metadata['tagline'].fillna('')

# Преобразование tagline в числовые признаки с использованием CountVectorizer
count_vectorizer = CountVectorizer(max_features=10000, stop_words='english')
tagline_features = count_vectorizer.fit_transform(movies_metadata['tagline'])

# Преобразование overview в TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
overview_tfidf = tfidf_vectorizer.fit_transform(movies_metadata['overview'].fillna(''))

# Получаем список уникальных жанров
all_genres = movies_metadata['genres'].dropna().explode().unique()

# Создаем словарь для индексации жанров
genre_to_index = {genre: idx for idx, genre in enumerate(all_genres)}

# Преобразуем жанры в индексы
movies_metadata['genre_indices'] = movies_metadata['genres'].apply(lambda genres: [genre_to_index[genre] for genre in genres])

# Подготовка числовых признаков
numerical_features = np.hstack([movies_metadata[['vote_average', 'popularity', 'vote_count']].values, tagline_features.toarray()])
final_features = np.hstack([numerical_features, overview_tfidf.toarray()])

In [7]:
# Ресэмплинг данных для ускорения кластеризации
X_sample = resample(final_features, n_samples=2000, random_state=42)

# Иерархическая кластеризация на подмножестве данных
agg_clustering = AgglomerativeClustering(distance_threshold=0.5, n_clusters=None)
clusters_sample = agg_clustering.fit_predict(X_sample)

# Получаем ссылки на кластеры из агломеративного дерева
linkage_matrix = agg_clustering.children_



In [8]:
# Шаг 2: Применяем ближайших соседей для всех точек X_train_encoded, чтобы найти кластер для каждой точки
nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(X_sample)
distances, indices = nbrs.kneighbors(final_features)

# Теперь каждому элементу из X_train_encoded мы присваиваем кластер ближайшего соседа из X_sample
clusters_all = clusters_sample[indices.flatten()]

# Добавляем кластеры в основной набор данных
movies_metadata['cluster'] = clusters_all

# Проверим количество фильмов в каждом кластере
print(movies_metadata['cluster'].value_counts())

# Выводим пример распределения кластеров
print(movies_metadata[['title', 'cluster']].head())

cluster
0       6809
2        177
510      114
85       100
1503      60
        ... 
1314       1
1529       1
477        1
1066       1
1661       1
Name: count, Length: 1793, dtype: int64
                         title  cluster
0                    Toy Story        0
1                      Jumanji       92
2             Grumpier Old Men       85
3            Waiting to Exhale     1083
4  Father of the Bride Part II     1411


In [21]:
# Функция для получения истинных рекомендаций (например, из ground_truth)
def get_true_recommendations(movie_id):
    return ground_truth.get(movie_id, [])
    
# Функция для получения рекомендаций по кластеру
def recommend_by_cluster(movie_id, num_recommendations=5):
    # Убедимся, что movie_id передается в нужном формате
    movie_id = movie_id.strip()  # Удаляем пробелы вокруг
    if movie_id not in movies_metadata['imdb_id'].values:
        print(f"Movie ID {movie_id} not found in movies_metadata.")
        return []  # Возвращаем пустой список, если не нашли
    # Получаем кластер
    cluster = movies_metadata.loc[movies_metadata['imdb_id'] == movie_id, 'cluster'].values[0]
    cluster_movies = movies_metadata[movies_metadata['cluster'] == cluster]
    recommended_movies = cluster_movies['imdb_id'].values[:num_recommendations]  # Рекомендуем фильмы из того же кластера
    return recommended_movies

# Оценка на ground_truth
y_true = []  # Истинные рекомендации
y_pred = []  # Предсказанные рекомендации

for movie_id in ground_truth:
    true_recommendations = get_true_recommendations(movie_id)  # Получаем реальные рекомендации
    predicted_recommendations = recommend_by_cluster(movie_id)  # Получаем рекомендованные фильмы по кластеру
    
    # Проверяем, что список рекомендованных фильмов не пуст
    if len(predicted_recommendations) > 0:
        # Заполняем список истинных и предсказанных рекомендаций
        y_true.extend([1 if imdb in true_recommendations else 0 for imdb in predicted_recommendations])  # бинаризация
        y_pred.extend([1 if imdb in predicted_recommendations else 0 for imdb in predicted_recommendations])

# Рассчитываем Precision, Recall и F1-Score
precision = precision_score(y_true, y_pred, average='micro', zero_division=0)
recall = recall_score(y_true, y_pred, average='micro', zero_division=0)
f1 = f1_score(y_true, y_pred, average='micro', zero_division=0)

print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-Score: {f1:.4f}')

Precision: 0.0045
Recall: 0.0045
F1-Score: 0.0045


In [22]:
def calculate_map(y_true, y_pred):
    return average_precision_score(y_true, y_pred, average='micro')

map_score = calculate_map(y_true, y_pred)
print(f'Mean Average Precision (MAP): {map_score:.4f}')


Mean Average Precision (MAP): 0.0045


In [23]:
def ndcg_at_k(y_true, y_pred, k=5):
    dcg = 0
    idcg = 0
    for i in range(k):
        if y_true[i] == 1:
            dcg += 1 / np.log2(i + 2)  # +2 because ranks are 1-based
        if sorted(y_true, reverse=True)[i] == 1:
            idcg += 1 / np.log2(i + 2)
    return dcg / idcg if idcg != 0 else 0

ndcg_score = ndcg_at_k(y_true, y_pred, k=5)
print(f'NDCG@5: {ndcg_score:.4f}')

NDCG@5: 0.0000
